# Úkol 2

In [ ]:
%pip install azure-eventhub pandas

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
from azure.eventhub import EventHubConsumerClient, EventHubProducerClient, EventData
import json
import time
from datetime import datetime, timedelta
from threading import Thread, Event

CONNECTION_STR = "******"
EVENT_HUB_NAME = "szcasestudykotyk"

In [ ]:
events_buffer = []
stop_event = Event()

def on_event(partition_context, event):
    try:
        data = json.loads(event.body_as_str())
        if data.get("typ") == "vstup":
            data["cas_vytvoreni"] = datetime.fromisoformat(data["cas_vytvoreni"])
            events_buffer.append(data)
    except Exception as e:
        print(e)


def aggregate_loop():
    producer = EventHubProducerClient.from_connection_string(
        conn_str=CONNECTION_STR,
        eventhub_name=EVENT_HUB_NAME
    )
    while not stop_event.is_set():
        now = datetime.utcnow()
        cutoff = now - timedelta(minutes=5)

        recent_events = [e for e in events_buffer if e["cas_vytvoreni"] >= cutoff]

        agregace = {}
        for e in recent_events:
            hra = e["id_hry"]
            agregace[hra] = agregace.get(hra, 0) + e["vyse_sazky"]

        for hra, soucet in agregace.items():
            result = {
                "cas_odeslani": now.isoformat(),
                "id_hry": hra,
                "soucet_sazek": round(soucet, 2),
                "typ": "vystup"
            }
            producer.send_batch([EventData(json.dumps(result))])
            print("Odesláno:", result)

        time.sleep(20)
    producer.close()

# Spuštění příjmu eventů
consumer = EventHubConsumerClient.from_connection_string(
    conn_str=CONNECTION_STR,
    consumer_group="$Default",
    eventhub_name=EVENT_HUB_NAME
)

try:

    Thread(target=aggregate_loop, daemon=True).start()

    with consumer:
        consumer.receive(
            on_event=on_event,
            starting_position="-1"
        )
except KeyboardInterrupt:
    stop_event.set()


Odesláno: {'cas_odeslani': '2025-08-05T17:54:33.453287', 'id_hry': 'Hra_1', 'soucet_sazek': 3232.06, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:54:33.453287', 'id_hry': 'Hra_3', 'soucet_sazek': 1315.06, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:54:33.453287', 'id_hry': 'Hra_2', 'soucet_sazek': 3359.85, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:54:54.206114', 'id_hry': 'Hra_1', 'soucet_sazek': 4708.04, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:54:54.206114', 'id_hry': 'Hra_3', 'soucet_sazek': 3727.61, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:54:54.206114', 'id_hry': 'Hra_2', 'soucet_sazek': 5788.12, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:55:14.277326', 'id_hry': 'Hra_1', 'soucet_sazek': 4803.0, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:55:14.277326', 'id_hry': 'Hra_3', 'soucet_sazek': 4502.22, 'typ': 'vystup'}
Odesláno: {'cas_odeslani': '2025-08-05T17:55:14.277326', 

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can